In [1]:
%load_ext blackcellmagic

In [2]:
from crawto.CrawtoDS import CrawtoDS

In [3]:
import pandas
df = pandas.read_csv("train.csv")

In [4]:
c = CrawtoDS(
    data=df,
    target="Survived",
    problem='classification'
)

In [7]:
c.tsne()